In [2]:
from ReliefF import ReliefF
import numpy as np
import text_normalizer as tn
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
file_info = "C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\cleaned_data_v1.csv"
df = pd.read_csv(file_info, sep='\t')

In [4]:
df.head()

,url,code,category,language,title,summary,key_words,content,combined,clean_combined
0,*://BLAZINHERB.CA,200,dr,en,Blazin Herb,The Diamond High Crew,Blazin herb,Blazin Herb THC Edibles Concentrates Flower/bu...,Blazin Herb The Diamond High Crew Blazin herb ...,blazin herb diamond high crew blazin herb blaz...
1,*://THEHONEYPOTSTORE.CA,200,dr,en,The Honey Pot Cannabis and Medical Marijuana p...,"The Honey Pot Hamilton Ontario, provides canna...","The Honey Pot ,Georgia Peach Cannabis and Medi...",The Honey Pot Home About Us F.A.Q.'s About The...,The Honey Pot Cannabis and Medical Marijuana p...,honey pot cannabis medical marijuana product s...
2,*://AMANTE.CA,200,dr,en,Amante Organics - Licensed Certified Organic C...,np-template-header-footer-from-plugin,"Organic Cannabis, Certified Organic, Weed, Org...",Amante Organics LOCAL | ORGANIC | SUSTAINABLE ...,Amante Organics - Licensed Certified Organic C...,amante organics licensed certified organic can...
3,*://CANNAFLY.CA,200,dr,en,Cannafly,Cannafly,"Marijuana, cannabis, medical marijuana, dispen...",Vancouver's Premium Mobile Medicinal Cannabis ...,"Cannafly Cannafly Marijuana, cannabis, medical...",cannafly cannafly marijuana cannabis medical m...
4,*://GRON.VC,200,dr,en,Grøn Ventures,We are backing businesses and brands with pote...,"cannabis vc, cannabis venture capital, gron ve...","GRØN VENTURES 4675 MacArthur Ct, 15th Floor, N...",Grøn Ventures We are backing businesses and br...,grn ventures back business brand potential mar...


In [5]:
from sklearn.model_selection import train_test_split
#get 10% holdout set
train_corpus, holdout_corpus, train_label, holdout_label = train_test_split(np.array(df['clean_combined']), 
                                                                            np.array(df['category']),
                                                                           test_size=.10,random_state=1234)

#get 80% train and 20% test sets
train_corpus, test_corpus, train_label, test_label = train_test_split(train_corpus,
                                                                      train_label,
                                                                      test_size=.2225,random_state=1234) #.9 * .2225 = .20

train_corpus.shape, test_corpus.shape, holdout_corpus.shape

((29302,), (8386,), (4188,))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

#build BOW features on train corpus
tv = TfidfVectorizer(use_idf=True, min_df=0.0,max_df=1.0)
tv_train_features = tv.fit_transform(train_corpus)
#transform test corpus into features
tv_test_features = tv.transform(test_corpus)
#transform holdout corpus into features
tv_holdout_features = tv.transform(holdout_corpus)

print('TFIDF model:> Train features shape:',tv_train_features.shape,'Test features shape:',tv_test_features.shape,'Holdout features shape:',tv_holdout_features.shape)

TFIDF model:> Train features shape: (29302, 364296) Test features shape: (8386, 364296) Holdout features shape: (4188, 364296)


In [24]:
#feature importance using random forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

sel = SelectFromModel(RandomForestClassifier(n_estimators = 100,random_state=1234))
sel.fit(tv_train_features, train_label)

SelectFromModel(estimator=RandomForestClassifier(random_state=1234))

In [37]:
tv.get_feature_names()[10000]


'alston'

In [48]:
#create a data frame that includes all features and whether or not they are deemed important by feature_selection
feat_imp = pd.DataFrame(list(zip(tv.get_feature_names(),sel.get_support().tolist())),columns = ['Features','Important'])


,Features,Important
0,aa,True
1,aaa,True
2,aaaa,True
3,aaaaa,False
4,aaaaaa,False


In [54]:
#Now we can grab the important features so that we can select only the important features from the sparse matrix created by TfidfVectorizer
imp_Feats_array = np.array(feat_imp[feat_imp.Important == True].index)

0             aa
1            aaa
2           aaaa
91           aac
260          aan
           ...  
364186    zwolle
364190        zx
364202        zy
364270       zyz
364272        zz
Name: Features, Length: 20182, dtype: object

In [64]:
#Now we can pull only the important features out of the original train, test, and holdout matrices
tv_train_features_sub = tv_train_features[:,imp_Feats_array]
tv_test_features_sub = tv_test_features[:,imp_Feats_array]
tv_holdout_features_sub = tv_holdout_features[:,imp_Feats_array]

#Examine the new shapes of the sub-selected features
print('TFIDF model:> Train features shape:',tv_train_features_sub.shape,'Test features shape:',tv_test_features_sub.shape,'Holdout features shape:',tv_holdout_features_sub.shape)

TFIDF model:> Train features shape: (29302, 20182) Test features shape: (8386, 20182) Holdout features shape: (4188, 20182)


In [65]:
#SVM
from sklearn.svm import LinearSVC

svm = LinearSVC(penalty='l2',C=1,random_state=1234)
svm.fit(tv_train_features_sub,train_label)

LinearSVC(C=1, random_state=1234)

In [67]:
svm_tfidf_cv_scores = cross_val_score(svm,tv_train_features_sub,train_label,cv=5)

In [68]:
svm_tfidf_cv_mean_score = np.mean(svm_tfidf_cv_scores)

In [69]:
print('CV Accuracy (5-fold):',svm_tfidf_cv_scores)
print('Mean CV Accuracy:',svm_tfidf_cv_mean_score)

CV Accuracy (5-fold): [0.83910595 0.84883126 0.85307167 0.84982935 0.84744027]
Mean CV Accuracy: 0.8476557018016356


In [70]:
svm_tfidf_test_score = svm.score(tv_test_features_sub,test_label)
print('Test Accuracy:',svm_tfidf_test_score)

Test Accuracy: 0.8560696398759838


In [71]:
import model_evaluation_utils as meu
svm_predictions = svm.predict(tv_test_features_sub)
meu.get_metrics(true_labels=test_label,predicted_labels=svm_predictions)

Accuracy: 0.8561
Precision: 0.8562
Recall: 0.8561
F1 Score: 0.856


In [72]:
unique_classes = list(set(test_label))
meu.display_classification_report(true_labels=test_label,predicted_labels=svm_predictions,classes=unique_classes)

              precision    recall  f1-score   support

          os       0.82      0.78      0.80      1393
          ed       0.90      0.90      0.90       783
          dr       0.98      0.93      0.96       307
          sp       0.89      0.89      0.89       494
          mk       0.75      0.75      0.75      1709
          sx       0.99      0.99      0.99      1345
          bu       0.84      0.87      0.85      2355

    accuracy                           0.86      8386
   macro avg       0.88      0.87      0.88      8386
weighted avg       0.86      0.86      0.86      8386



In [73]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfc = RandomForestClassifier(n_estimators=10,random_state=1234)
rfc.fit(tv_train_features_sub,train_label)


RandomForestClassifier(n_estimators=10, random_state=1234)

In [74]:
rfc_tfidf_cv_scores = cross_val_score(rfc,tv_train_features_sub,train_label,cv=5)

In [75]:
rfc_tfidf_cv_mean_score = np.mean(rfc_tfidf_cv_scores)

In [76]:
print('CV Accuracy (5-fold):',rfc_tfidf_cv_scores)
print('Mean CV Accuracy:',rfc_tfidf_cv_mean_score)

CV Accuracy (5-fold): [0.75874424 0.75482    0.75989761 0.77440273 0.77372014]
Mean CV Accuracy: 0.7643169432000619


In [77]:
rfc_tfidf_test_score = rfc.score(tv_test_features_sub,test_label)
print('Test Accuracy:',rfc_tfidf_test_score)

Test Accuracy: 0.7804674457429048


In [78]:
import model_evaluation_utils as meu
rfc_predictions = rfc.predict(tv_test_features_sub)
meu.get_metrics(true_labels=test_label,predicted_labels=svm_predictions)

Accuracy: 0.7805
Precision: 0.7895
Recall: 0.7805
F1 Score: 0.7803


In [79]:
unique_classes = list(set(test_label))
meu.display_classification_report(true_labels=test_label,predicted_labels=rfc_predictions,classes=unique_classes)

              precision    recall  f1-score   support

          os       0.80      0.71      0.76      1393
          ed       0.87      0.73      0.79       783
          dr       0.99      0.85      0.91       307
          sp       0.91      0.68      0.77       494
          mk       0.64      0.61      0.63      1709
          sx       0.99      0.98      0.98      1345
          bu       0.70      0.86      0.77      2355

    accuracy                           0.78      8386
   macro avg       0.84      0.77      0.80      8386
weighted avg       0.79      0.78      0.78      8386



In [80]:
#Let's check Random Forest using more estimators for comparison
rfc = RandomForestClassifier(n_estimators=100,random_state=1234)
rfc.fit(tv_train_features_sub,train_label)

RandomForestClassifier(random_state=1234)

In [81]:
rfc_tfidf_cv_scores = cross_val_score(rfc,tv_train_features_sub,train_label,cv=5)

In [82]:
rfc_tfidf_cv_mean_score = np.mean(rfc_tfidf_cv_scores)

In [83]:
print('CV Accuracy (5-fold):',rfc_tfidf_cv_scores)
print('Mean CV Accuracy:',rfc_tfidf_cv_mean_score)

CV Accuracy (5-fold): [0.82255588 0.82886879 0.828157   0.83242321 0.83037543]
Mean CV Accuracy: 0.8284760605914144


In [84]:
rfc_tfidf_test_score = rfc.score(tv_test_features_sub,test_label)
print('Test Accuracy:',rfc_tfidf_test_score)

Test Accuracy: 0.8433102790364894


In [85]:
import model_evaluation_utils as meu
rfc_predictions = rfc.predict(tv_test_features_sub)
meu.get_metrics(true_labels=test_label,predicted_labels=rfc_predictions)

Accuracy: 0.8433
Precision: 0.8492
Recall: 0.8433
F1 Score: 0.8435


In [86]:
unique_classes = list(set(test_label))
meu.display_classification_report(true_labels=test_label,predicted_labels=rfc_predictions,classes=unique_classes)

              precision    recall  f1-score   support

          os       0.84      0.79      0.81      1393
          ed       0.94      0.79      0.86       783
          dr       1.00      0.86      0.92       307
          sp       0.94      0.76      0.84       494
          mk       0.74      0.73      0.74      1709
          sx       0.99      0.99      0.99      1345
          bu       0.78      0.91      0.84      2355

    accuracy                           0.84      8386
   macro avg       0.89      0.83      0.86      8386
weighted avg       0.85      0.84      0.84      8386

